## Just the LLM (No Tools , No Agent)

In [5]:
from openai import OpenAI
from IPython.display import Markdown, display
client = OpenAI()

def simple_llm(question: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question},
        ],
    )
    return display(Markdown(response.choices[0].message.content))

print(simple_llm("Give me 3 budget spots to visit in Cape Town"))

Cape Town is a beautiful city with plenty of budget-friendly options for visitors. Here are three great budget spots to consider:

1. **Table Mountain**:
   - While taking the cable car to the top can be pricey, hiking up Table Mountain is a free and rewarding experience. There are several trails such as the Platteklip Gorge route that are well-marked and offer breathtaking views of the city and coastline. Remember to pack water and snacks for your hike!

2. **Boulders Beach**:
   - Famous for its resident African penguin colony, Boulders Beach is a unique spot you can visit on a budget. While there is a small entrance fee to access certain areas of the beach, you can still view the penguins from the boardwalk for free. You can enjoy the beach, take pictures, and have a picnic without spending much.

3. **The Company's Garden**:
   - Located in the heart of the city, the Company's Garden is a peaceful public park that is free to enter. It features beautiful gardens, squirrels, and historical monuments. You can stroll through the garden, visit the nearby South African Museum, or relax on the lawns. It's a great spot for a leisurely afternoon without breaking the bank.

These spots give you a taste of Cape Town's natural beauty and local culture without requiring a big budget. Enjoy your visit!

None


## Add a Tool (a function the model can use)

In [13]:
from openai import OpenAI
import json
import math

client = OpenAI()

def calculate_square_root(x: float) -> float:
    return math.sqrt(x)

tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_square_root",
            "description": "Calculate the square root of a number.",
            "parameters": {
                "type": "object",
                "properties": {
                    "x": {"type": "number"},
                },
                "required": ["x"],
            },
        },
    }
]


def math_with_tool(question: str) -> str:
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a math helper. Use the tool if needed."},
            {"role": "user", "content": question},
        ],
        tools = tools,
    )
    
    message = response.choices[0].message

    if message.tool_calls:
        tool_call = message.tool_calls[0]
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)

        if tool_name == "calculate_square_root":
            result = calculate_square_root(**tool_args)

            follow_up = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Explain the result clearly for the user."},
                    {"role": "user", "content": question},
                    message,
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": str(result),
                    },
                ],
            )
            return follow_up.choices[0].message.content
    return message.content

print(math_with_tool("What is the square root of 16?"))



The square root of 16 is 4. This means that when you multiply 4 by itself (4 × 4), you get 16.
